In [1]:
import pandas as pd
from datetime import datetime

In [2]:
airports = pd.read_csv('data/airports.csv')
airports.columns = airports.columns.str.lower()
airports.head()

,iata_code,airport,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [3]:
flights = pd.read_csv('data/flights.csv')

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
flights.columns = flights.columns.str.lower()

flights = flights.merge(right=airports, how='left', left_on='origin_airport', right_on='iata_code')
flights.rename(columns={'latitude': 'origin_latitude', 'longitude': 'origin_longitude'}, inplace=True)

flights = flights.merge(right=airports, 
                        how='left', left_on='destination_airport', right_on='iata_code',
                        suffixes=('_origin', '_destination'))
flights.head()

,year,month,day,day_of_week,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,...,country_origin,origin_latitude,origin_longitude,iata_code_destination,airport_destination,city_destination,state_destination,country_destination,latitude,longitude
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [5]:
flights_clean = flights[flights.columns.difference([
    'year', # all are 2015
    'flight_number', # irrelevant
    'tail_number', # irrelevant
    'departure_time', # collinearity: scheduled_departure + departure_delay
    'wheels_off', # co: departure_time + taxi_out
    'elapsed_time', # co: scheduled_time + departure_delay
    'air_time', # co: elapsed_time - taxi_out - taxi_in
    'wheels_on', # co: arrival_time - taxi_in
    'arrival_time', # co: scheduled_arrival
    # leave in scheduled_arrival because of time change
    'arrival_delay', # co: departure_delay
    'diverted', # occurs after delay, can't be used to predict delay time
    'cancelled', # occurs after delay
    'cancellation_reason',
    'air_system_delay',
    'security_delay',
    'airline_delay',
    'late_aircraft_delay',
    'weather_delay',
    'iata_code_origin', 'iata_code_destination', # same as origin_airport
    'airport_origin', 'airport_destination', # airport already described in origin_airport
    'country_origin', 'country_destination' # all are USA
])]

In [6]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,distance,latitude,longitude,...,origin_airport,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,1448,47.44898,-122.30931,...,ANC,61.17432,-149.99619,430,5,205.0,WA,AK,4.0,21.0
1,AA,West Palm Beach,Los Angeles,1,4,-8.0,PBI,2330,26.68316,-80.09559,...,LAX,33.94254,-118.40807,750,10,280.0,FL,CA,4.0,12.0
2,US,Charlotte,San Francisco,1,4,-2.0,CLT,2296,35.21401,-80.94313,...,SFO,37.61900,-122.37484,806,20,286.0,NC,CA,11.0,16.0
3,AA,Miami,Los Angeles,1,4,-5.0,MIA,2342,25.79325,-80.29056,...,LAX,33.94254,-118.40807,805,20,285.0,FL,CA,8.0,15.0
4,AS,Anchorage,Seattle,1,4,-1.0,ANC,1448,61.17432,-149.99619,...,SEA,47.44898,-122.30931,320,25,235.0,AK,WA,5.0,11.0


In [7]:
def time_to_minutes(n):
    # converts from HHMM time (written as number)
    # to number of minutes elapsed since midnight
    return (n // 100) * 60 + (n % 100)

In [8]:
flights_clean.scheduled_departure = flights_clean.scheduled_departure.apply(time_to_minutes)
flights_clean.scheduled_arrival = flights_clean.scheduled_arrival.apply(time_to_minutes)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,distance,latitude,longitude,...,origin_airport,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,1448,47.44898,-122.30931,...,ANC,61.17432,-149.99619,270,5,205.0,WA,AK,4.0,21.0
1,AA,West Palm Beach,Los Angeles,1,4,-8.0,PBI,2330,26.68316,-80.09559,...,LAX,33.94254,-118.40807,470,10,280.0,FL,CA,4.0,12.0
2,US,Charlotte,San Francisco,1,4,-2.0,CLT,2296,35.21401,-80.94313,...,SFO,37.61900,-122.37484,486,20,286.0,NC,CA,11.0,16.0
3,AA,Miami,Los Angeles,1,4,-5.0,MIA,2342,25.79325,-80.29056,...,LAX,33.94254,-118.40807,485,20,285.0,FL,CA,8.0,15.0
4,AS,Anchorage,Seattle,1,4,-1.0,ANC,1448,61.17432,-149.99619,...,SEA,47.44898,-122.30931,200,25,235.0,AK,WA,5.0,11.0


In [10]:
def day_counter(row):
    # convert from month and day to nth day of year (i.e. feb 1 -> 32)
    return (datetime(year=2015, month=row.month, day=row.day) - datetime(year=2014, month=12, day=31)).days

In [11]:
flights_clean['day_of_year'] = flights_clean.apply(day_counter, axis=1)

<ipython-input-11-378aa3fec5ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_clean['day_of_year'] = flights_clean.apply(day_counter, axis=1)


In [12]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,distance,latitude,longitude,...,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,1448,47.44898,-122.30931,...,61.17432,-149.99619,270,5,205.0,WA,AK,4.0,21.0,1
1,AA,West Palm Beach,Los Angeles,1,4,-8.0,PBI,2330,26.68316,-80.09559,...,33.94254,-118.40807,470,10,280.0,FL,CA,4.0,12.0,1
2,US,Charlotte,San Francisco,1,4,-2.0,CLT,2296,35.21401,-80.94313,...,37.61900,-122.37484,486,20,286.0,NC,CA,11.0,16.0,1
3,AA,Miami,Los Angeles,1,4,-5.0,MIA,2342,25.79325,-80.29056,...,33.94254,-118.40807,485,20,285.0,FL,CA,8.0,15.0,1
4,AS,Anchorage,Seattle,1,4,-1.0,ANC,1448,61.17432,-149.99619,...,47.44898,-122.30931,200,25,235.0,AK,WA,5.0,11.0,1


In [13]:
flights_clean.loc[0]

airline                       AS
city_destination         Seattle
city_origin            Anchorage
day                            1
day_of_week                    4
departure_delay            -11.0
destination_airport          SEA
distance                    1448
latitude                47.44898
longitude             -122.30931
month                          1
origin_airport               ANC
origin_latitude         61.17432
origin_longitude      -149.99619
scheduled_arrival            270
scheduled_departure            5
scheduled_time             205.0
state_destination             WA
state_origin                  AK
taxi_in                      4.0
taxi_out                    21.0
day_of_year                    1
Name: 0, dtype: object

In [14]:
flights_clean.dtypes

airline                 object
city_destination        object
city_origin             object
day                      int64
day_of_week              int64
departure_delay        float64
destination_airport     object
distance                 int64
latitude               float64
longitude              float64
month                    int64
origin_airport          object
origin_latitude        float64
origin_longitude       float64
scheduled_arrival        int64
scheduled_departure      int64
scheduled_time         float64
state_destination       object
state_origin            object
taxi_in                float64
taxi_out               float64
day_of_year              int64
dtype: object

In [15]:
flights_clean.to_csv('data/flights_clean.csv', index=False)